# Self-Driving Car Engineer Nanodegree


## Project: **Advanced Lane finding ** 

### Part 1: Camera Calibration + Writeup preparation

#### Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML

from line import *
from support_functions import *

%matplotlib inline

#### Read saved parameters and add new parameters:

In [2]:
calibration_pickle = pickle.load( open( "calibration_parameters.p", "rb" ) )
mtx = calibration_pickle["mtx"]
dist = calibration_pickle["dist_coff"]
M_mean = calibration_pickle["matrix"]
MInverse = calibration_pickle["inverse_matrix"]
s_channel_thresh = calibration_pickle["s_channel_thresh"]
gradient_thresh = calibration_pickle["gradient_thresh"]
l_channel_thresh = calibration_pickle["l_channel_thresh"]

In [3]:
#n-
#search from previous margin
search_margin = 60
# number of sliding windows
nwindows = 9
# Set the width of the windows +/- margin
window_margin = 30
# Set minimum number of pixels found to recenter window
minpix = 40
# Define conversions in x and y from pixels space to meters
ym_per_pix = (30/720) # meters per pixel in y dimension
xm_per_pix = (3.7/700) # meters per pixel in x dimension


In [4]:
my_Lane = Lane(search_margin,nwindows,window_margin,minpix)


In [5]:
def process_image(image):
    #get image size: 
    imgsize = (image.shape[1] , image.shape[0])
    
    #undistort the input image :
    undist= cv2.undistort(image, mtx, dist, None, mtx)
    
    #prospective transform
    warped =cv2.warpPerspective(undist, M_mean, imgsize)
    
    #color and gradient transform filter
    filtered_binary = lane_filter(warped,s_channel_thresh,l_channel_thresh,gradient_thresh)
    
    #call to fit polynomial, highlight lane  and write text 
    lane_highlight_wraped,curve_text,vehicle_offset_text = my_Lane.get_lane_highlighted(filtered_binary,ym_per_pix,xm_per_pix)
    lane_highlight_unwraped =cv2.warpPerspective(lane_highlight_wraped, MInverse, imgsize)

    #add overlay with the lane
    image = cv2.addWeighted(undist, 1, lane_highlight_unwraped, 0.3, 0)
     
    #add text on image
    cv2.putText(image, curve_text, (20, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(image, vehicle_offset_text, (20, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
    
    
    return image

#### Read Video and pass to pipline 

In [6]:
project_output = 'output_videos/project_video_output.mp4'
clip1 = VideoFileClip("test_videos/project_video.mp4")#.subclip(38,42)
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

t:   0%|          | 2/1260 [00:00<01:20, 15.55it/s, now=None]

Moviepy - Building video output_videos/project_video_output.mp4.
Moviepy - Writing video output_videos/project_video_output.mp4



Moviepy - Done !
Moviepy - video ready output_videos/project_video_output.mp4
Wall time: 4min 17s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))